In [1]:
# Import Julia packages
using DrWatson
@quickactivate
using Revise
using LinearAlgebra
using DelimitedFiles
using SparseArrays
using StaticArrays
using BlockArrays
using CairoMakie
using UnPack
using FromFile
using GeometryBasics
using Random
using Colors
using JLD2
using LaTeXStrings
using Glob
using Printf


@from "$(projectdir())/src/VertexModelContainers.jl" using VertexModelContainers
@from "$(projectdir())/src/Laplacians.jl" using Laplacians
@from "$(projectdir())/src/AnalysisFunctions.jl" using AnalysisFunctions
@from "$(projectdir())/src/CellProperties.jl" using CellProperties

In [ ]:
f=Glob.glob("relax_100_cells/*L₀=3.99*","C:\\Users\\v35431nc\\Documents\\VM_code\\VertexModel\\data\\sims" )[1]

In [ ]:
@unpack R, matrices, params = load(datadir(f,"frameData/systemDataFinal.jld2"))
@unpack B, Bᵀ, C, cellPositions, cellAreas, cellPerimeters = matrices
@unpack nCells,nVerts = params
mkpath(datadir(f,"eigenmodes"))
nCells

In [31]:
G=makeG(params)

M=makeM(matrices)

In [ ]:
Lc=makeEvLc(M)

In [ ]:
emodes=eigen(Matrix(Lc*G))
Lcevals=real(emodes.values)
Lcevecs=Matrix(qr(real(emodes.vectors)).Q)


In [ ]:
writedlm(datadir(f,"eigenmodes","eigenvalues_LG.csv"), Lcevals, ',') 
writedlm(datadir(f,"eigenmodes","eigenvectors_LG.csv"), Lcevecs, ',')
writedlm(datadir(f,"eigenmodes","Lc.csv"), Lc, ',') 

In [ ]:
Lv=makeEvLv(M, G)

In [ ]:
Lvevals,Lvevecs=LAPACK.syev!('V','U',deepcopy(Matrix(mortar(Lv))))
writedlm(datadir(f,"eigenmodes","eigenvalues_MTGM.csv"), Lvevals, ',') 
writedlm(datadir(f,"eigenmodes","eigenvectors_MTGM.csv"), Lvevecs, ',')  

In [ ]:
X=makeX(params, matrices)

@unpack cellPressures, cellTensions =matrices
g=vcat(cellPressures, -cellTensions)
gX=Matrix{SMatrix{2,2,Float64,4}}(undef,nVerts,nVerts)
fill!(gX,@SMatrix zeros(2,2))
for α=1:2*nCells
    gX+=g[α]X[α, :,:]
end

In [ ]:
DDpp=makeD(params,matrices, X, Lvevals, Lvevecs, 2*nCells-1)
DDppevals,DDppevecs=LAPACK.syev!('V','U',deepcopy(DDpp))

# writedlm(datadir(f,"eigenmodes","Dpp_diag_sigma_eigenvalues.csv"), DDppevals, ',') 

# writedlm(datadir(f,"eigenmodes","Dpp_diag_sigma_eigenvectors.csv"), DDppevecs, ',') 

In [ ]:
@unpack cellTensions, cellPressures = matrices
@unpack nCells, nVerts = params
q=2*nCells-1
g=vcat(cellPressures, -cellTensions)
gX=Matrix{SMatrix{2,2,Float64,4}}(undef,nVerts,nVerts)
fill!(gX,@SMatrix zeros(2,2))
for α=1:2*nCells
    gX+=g[α]X[α, :,:]
end

D=zeros(2*nVerts, 2*nVerts)
qeval=Lvevals[2*nVerts+1-q: 2*nVerts]

D=Lvevecs'*Matrix(mortar(gX))*Lvevecs
for i=1:q
    D[(2*nVerts-q)+i,(2*nVerts-q)+i]+=qeval[i]
end

In [ ]:
DDpptotevals,DDpptotevecs=LAPACK.syev!('V','U',deepcopy(D))

writedlm(datadir(f,"eigenmodes","Dpp_tot_diag_sigma_eigenvalues.csv"), DDpptotevals, ',') 

writedlm(datadir(f,"eigenmodes","Dpp_tot_diag_sigma_eigenvectors.csv"), DDpptotevecs, ',') 

In [ ]:
H=Matrix(mortar(Lv)).+Matrix(mortar(gX))
evalH, evecH=eigen(H)
evecH=Matrix(qr(evecH).Q)

writedlm(datadir(f,"eigenmodes","H_eigenvalues.csv"), evalH, ',') 

writedlm(datadir(f,"eigenmodes","H_eigenvectors.csv"), evecH, ',') 

In [ ]:
LAPACK.syev!('V','U',deepcopy(H))


In [ ]:
n=LinRange(1, 2*nVerts, 2*nVerts)
fig = Figure()
ax=Axis(fig[1, 1], xlabel="n", ylabel="log₁₀(λₙ)", title="log₁₀(eigenvalues Lv,  MᵀGM+gX )")

scatter!(ax,n, log10.(abs.(Lvevals)), color=:black, label="MᵀGM", markersize=5)
scatter!(ax,n, log10.(abs.(evalH)), color=:blue, label="MᵀGM+gX", markersize=5)
scatter!(ax,n, log10.(abs.(DDpptotevals)), color=:orange, label="D_total+diag(σ²)", markersize=5)
vlines!(ax,2*nVerts-(2*nCells) +0.5, color=:red)
vlines!(ax,2*nVerts-(nCells) +0.5, color=:red)
fig[1, 2] = Legend(fig, ax, framevisible = false)
#save(datadir(f,"eigenmodes","Lv_MTGMgX_D_Evals.png"),fig)
fig

In [ ]:
n

In [ ]:
nv=LinRange(1, 2*nVerts, 2*nVerts)
nc=LinRange(2*nVerts-2*nCells+1, 2*nVerts, 2*nCells)
nD=LinRange(2*nVerts-2*nCells, 2*nVerts, 2*nCells-1)
fig = Figure()
ax=Axis(fig[1, 1], xlabel="n", ylabel="λₙ", title="Comparison of eigenvalues")

scatter!(ax,nv, log10.(sort(abs.(Lvevals))), color=:black,markersize=4, label="MᵀGM")
scatter!(ax,nv, log10.(sort(abs.(evalH))), color=:blue,markersize=4, label="MᵀGM+gX")
scatter!(ax,nc, log10.(sort(abs.(Lcevals))), color=:red,markersize=4, label="LG")
scatter!(ax,nD, log10.(sort(abs.(DDppevals))), color=:green,markersize=4, label="D+diag(σ²)")
scatter!(ax,n, log10.(sort(abs.(DDpptotevals))), color=:orange, label="Dtotal+diag(σ²)", markersize=4)
#vlines!(ax,2*nVerts+1-((2*nCells)-1), color=:red)
fig[1, 2] = Legend(fig, ax, framevisible = false)
save(datadir(f,"eigenmodes","compare_eigenvalues.png"),fig)
fig

In [ ]:
@unpack edgeTangents,A, Ā= matrices
@unpack nEdges = params

vertexAreas=zeros(nVerts)
edgeMidpointLinks=fill(SVector{2, Float64}(zeros(2)), (nCells, nVerts))

nzC = findnz(C)
ikPairs = tuple.(nzC[1],nzC[2])
for (i,k) in ikPairs
    for j=1:nEdges
        edgeMidpointLinks[i,k] = edgeMidpointLinks[i,k] .+ 0.5.*B[i,j]*edgeTangents[j]*Ā[j,k]
    end
end


##Check k_is length ==1

for k=1:nVerts

    k_is = findall(x->x!=0, C[:,k])
    if length(k_is) == 1
        edgesSharedBy_i1_And_k = findall(x->x!=0, B[k_is[1],:])∩findall(x->x!=0, A[:,k])
        vertexAreas[k] = 0.5^3*norm([edgeTangents[edgesSharedBy_i1_And_k[1]]...,0.0]×[edgeTangents[edgesSharedBy_i1_And_k[2]]...,0.0])
    elseif length(k_is) == 2
        edgesSharedBy_i1_And_k = findall(x->x!=0, B[k_is[1],:])∩findall(x->x!=0, A[:,k])
        vertexAreas[k] = 0.5^3*norm([edgeTangents[edgesSharedBy_i1_And_k[1]]...,0.0]×[edgeTangents[edgesSharedBy_i1_And_k[2]]...,0.0])
        edgesSharedBy_i2_And_k = findall(x->x!=0, B[k_is[2],:])∩findall(x->x!=0, A[:,k])
        vertexAreas[k] += 0.5^3*norm([edgeTangents[edgesSharedBy_i2_And_k[1]]...,0.0]×[edgeTangents[edgesSharedBy_i2_And_k[2]]...,0.0])
    else
        vertexAreas[k] = 0.5*norm([edgeMidpointLinks[k_is[1], k]...,0.0]×[edgeMidpointLinks[k_is[2],k]...,0.0])
    end
end

In [ ]:
E=Diagonal(vertexAreas)
LvE=inv(E)*Lv
LvEev=eigen(Matrix(mortar(LvE)))
LvEevals=real(LvEev.values)
LvEevecs=qr(LvEev.vectors).Q
LcE=tr.((M*inv(E))*M')
LcEev=eigen(Matrix(LcE*G))
LcEevals=LcEev.values
LcEevecs=qr(LcEev.vectors).Q

DE=zeros(2*nVerts, 2*nVerts)
q=2*nCells-1
qeval=LvEevals[2*nVerts+1-q: 2*nVerts]

DE=LvEevecs'*Matrix(mortar(gX))*LvEevecs
for i=1:q
    DE[(2*nVerts-q)+i,(2*nVerts-q)+i]+=qeval[i]
end
DDpptotEevals,DDpptotEevecs=LAPACK.syev!('V','U',deepcopy(DE))

HE=Matrix(mortar(LvE)).+Matrix(mortar(gX))
evalHE, evecHE=eigen(HE)
evecHE=Matrix(qr(real(evecHE)).Q)
evalHE=real(evalHE)

In [ ]:
DDppE=makeD(params,matrices, X, LvEevals, LvEevecs, 2*nCells-1)
DDppEevals,DDppEevecs=LAPACK.syev!('V','U',deepcopy(DDppE))

In [ ]:
writedlm(datadir(f,"eigenmodes","LcE_eigenvalues.csv"), LcEevals, ',') 
writedlm(datadir(f,"eigenmodes","LcE_eigenvectors.csv"), LcEevecs, ',') 
writedlm(datadir(f,"eigenmodes","LvE_eigenvalues.csv"), LvEevals, ',') 
writedlm(datadir(f,"eigenmodes","LvE_eigenvectors.csv"), LvEevecs, ',') 
writedlm(datadir(f,"eigenmodes","HE_eigenvalues.csv"), evalHE, ',') 
writedlm(datadir(f,"eigenmodes","HE_eigenvectors.csv"), evecHE, ',') 
writedlm(datadir(f,"eigenmodes","DppE_tot_diag_sigma_eigenvalues.csv"), DDpptotEevals, ',') 
writedlm(datadir(f,"eigenmodes","DppE_tot_diag_sigma_eigenvectors.csv"), DDpptotEevecs, ',') 

In [ ]:
nv=LinRange(1, 2*nVerts, 2*nVerts)
nc=LinRange(2*nVerts-2*nCells+1, 2*nVerts, 2*nCells)
nD=LinRange(2*nVerts-2*nCells, 2*nVerts, 2*nCells-1)
fig = Figure()
ax=Axis(fig[1, 1], xlabel="n", ylabel="λₙ", title="Comparison of eigenvalues")

scatter!(ax,nv, log10.(sort(abs.(LvEevals))), color=:black,markersize=4, label="E⁻¹MᵀGM")
scatter!(ax,nv, log10.(sort(abs.(evalHE))), color=:blue,markersize=4, label="E⁻¹MᵀGM+gX")
scatter!(ax,nc, log10.(sort(abs.(LcEevals))), color=:red,markersize=4, label="ME⁻¹.MᵀG")
#scatter!(ax,nD, log10.(sort(abs.(DDppEevals))), color=:green,markersize=4, label="D+diag(σ²)")
scatter!(ax,nv, log10.(sort(abs.(DDpptotEevals))), color=:orange, label="Dtotal+diag(σ²)", markersize=4)
#vlines!(ax,2*nVerts+1-((2*nCells)-1), color=:red)
fig[1, 2] = Legend(fig, ax, framevisible = false)
save(datadir(f,"eigenmodes","compare_eigenvalues_E.png"),fig)
fig

In [ ]:
cellPolygons = makeCellPolygons(R,params,matrices)

In [ ]:
for n=1:2*nCells
    Aevlims=(-maximum(abs.(Lcevecs[1:nCells,n])), maximum(abs.(Lcevecs[1:nCells, n])))
    Levlims=(-maximum(abs.(Lcevecs[ nCells+1:2*nCells,n])), maximum(abs.(Lcevecs[nCells+1:2*nCells, n])))
    set_theme!(figure_padding=1, backgroundcolor=(:white,1.0), font="Helvetica", fontsize=19)
    fig = Figure(resolution=(1500,500))

    a1=Axis(fig[1,1],aspect=DataAspect())
    a2=Axis(fig[2,1],aspect=DataAspect())
    hidedecorations!(a1)
    hidespines!(a1)
    hidedecorations!(a2)
    hidespines!(a2)
    for i=1:nCells
        poly!(a1,cellPolygons[i],color=[Lcevecs[1:nCells,n][i]],colormap=:bwr,colorrange=Aevlims, strokecolor=(:black,1.0),strokewidth=1)
        poly!(a2,cellPolygons[i],color=[Lcevecs[nCells+1:2*nCells,n][i]],colormap=:bwr,colorrange=Levlims, strokecolor=(:black,1.0),strokewidth=1)
    end
    Label(fig[2,1,Bottom()],"λ_"*string(n)*" = "*@sprintf("%.5E", Lcevals[n]),fontsize = 32)

    #hidedecorations!(ax22)
    #hidespines!(ax22)

    colsize!(fig.layout,1,Aspect(1,1.0))


    Colorbar(fig[1,2],limits=colorrange=Aevlims,colormap=:bwr,flipaxis=true)
    Colorbar(fig[2,2],limits=colorrange=Levlims,colormap=:bwr,flipaxis=true)


    Label(fig[1,1,Left()],string(L"Area"),fontsize = 32, rotation=π/2)
    Label(fig[2,1,Left()],string(L"Perimeter"),fontsize = 32, rotation=π/2)
    #Label( fig[0,:],"Γ = "*string(params.γ)*", L₀ = "*string(params.L₀)*", δL = "*string(params.δL),fontsize = 32, color = (:black, 1))
    resize_to_layout!(fig)
    save(datadir(f,"eigenmodes","eigenmodes_Lc$(@sprintf("%03d", n)).png"),fig)

    #display(fig)
end

In [ ]:
for n=1:2*nCells
    Aevlims=(-maximum(abs.(LcEevecs[1:nCells,n])), maximum(abs.(LcEevecs[1:nCells, n])))
    Levlims=(-maximum(abs.(LcEevecs[ nCells+1:2*nCells,n])), maximum(abs.(LcEevecs[nCells+1:2*nCells, n])))
    set_theme!(figure_padding=1, backgroundcolor=(:white,1.0), font="Helvetica", fontsize=19)
    fig = Figure(resolution=(1500,500))

    a1=Axis(fig[1,1],aspect=DataAspect())
    a2=Axis(fig[2,1],aspect=DataAspect())
    hidedecorations!(a1)
    hidespines!(a1)
    hidedecorations!(a2)
    hidespines!(a2)
    for i=1:nCells
        poly!(a1,cellPolygons[i],color=[LcEevecs[1:nCells,n][i]],colormap=:bwr,colorrange=Aevlims, strokecolor=(:black,1.0),strokewidth=1)
        poly!(a2,cellPolygons[i],color=[LcEevecs[nCells+1:2*nCells,n][i]],colormap=:bwr,colorrange=Levlims, strokecolor=(:black,1.0),strokewidth=1)
    end
    Label(fig[2,1,Bottom()],"λ_"*string(n)*" = "*@sprintf("%.5E", LcEevals[n]),fontsize = 32)

    #hidedecorations!(ax22)
    #hidespines!(ax22)

    colsize!(fig.layout,1,Aspect(1,1.0))


    Colorbar(fig[1,2],limits=colorrange=Aevlims,colormap=:bwr,flipaxis=true)
    Colorbar(fig[2,2],limits=colorrange=Levlims,colormap=:bwr,flipaxis=true)


    Label(fig[1,1,Left()],string(L"Area"),fontsize = 32, rotation=π/2)
    Label(fig[2,1,Left()],string(L"Perimeter"),fontsize = 32, rotation=π/2)
    #Label( fig[0,:],"Γ = "*string(params.γ)*", L₀ = "*string(params.L₀)*", δL = "*string(params.δL),fontsize = 32, color = (:black, 1))
    resize_to_layout!(fig)
    save(datadir(f,"eigenmodes","eigenmodes_LcE$(@sprintf("%03d", n)).png"),fig)

    #display(fig)
end

In [ ]:

cellPolygons = makeCellPolygons(R,params,matrices)
@unpack cellAreas, cellPerimeters = matrices
Aevlims=(minimum(abs.(cellAreas[1:nCells])), maximum(abs.(cellAreas[1:nCells])))
Levlims=(minimum(abs.(cellPerimeters[1:nCells])), maximum(abs.(cellPerimeters[1:nCells])))
set_theme!(figure_padding=1, backgroundcolor=(:white,1.0), font="Helvetica", fontsize=19)
fig = Figure(resolution=(1500,500))

a1=Axis(fig[1,1],aspect=DataAspect())
a2=Axis(fig[2,1],aspect=DataAspect())
hidedecorations!(a1)
hidespines!(a1)
hidedecorations!(a2)
hidespines!(a2)
for i=1:nCells
    poly!(a1,cellPolygons[i],color=cellAreas[i],colormap=:viridis,colorrange=Aevlims, strokecolor=(:black,1.0),strokewidth=1)
    poly!(a2,cellPolygons[i],color=cellPerimeters[i],colormap=:viridis,colorrange=Levlims, strokecolor=(:black,1.0),strokewidth=1)
end
#Label(fig[2,1,Bottom()],"λ_"*string(n)*" = "*@sprintf("%.5E", evals[n]),fontsize = 32)

#hidedecorations!(ax22)
#hidespines!(ax22)

colsize!(fig.layout,1,Aspect(1,1.0))


Colorbar(fig[1,2],limits=colorrange=Aevlims,colormap=:viridis,flipaxis=true)
Colorbar(fig[2,2],limits=colorrange=Levlims,colormap=:viridis,flipaxis=true)


Label(fig[1,1,Bottom()],string(L"Area"),fontsize = 32, rotation=0)
Label(fig[2,1,Bottom()],string(L"Perimeter"),fontsize = 32, rotation=0)
#Label( fig[0,:],"Γ = "*string(params.γ)*", L₀ = "*string(params.L₀)*", δL = "*string(params.δL),fontsize = 32, color = (:black, 1))
resize_to_layout!(fig)
save(datadir(f,"eigenmodes","Area_Perimeter.png"),fig)

display(fig)

In [ ]:
@unpack cellAreas, cellPerimeters, cellPressures, cellTensions = matrices
Aevlims=(minimum(cellPressures[1:nCells]), maximum(cellPressures[1:nCells]))
Levlims=(minimum(-cellTensions[1:nCells]), maximum(-cellTensions[1:nCells]))
set_theme!(figure_padding=1, backgroundcolor=(:white,1.0), font="Helvetica", fontsize=19)
fig = Figure(resolution=(1500,500))

a1=Axis(fig[1,1],aspect=DataAspect())
a2=Axis(fig[2,1],aspect=DataAspect())
hidedecorations!(a1)
hidespines!(a1)
hidedecorations!(a2)
hidespines!(a2)
for i=1:nCells
    poly!(a1,cellPolygons[i],color=cellPressures[i],colormap=:plasma,colorrange=Aevlims, strokecolor=(:black,1.0),strokewidth=1)
    poly!(a2,cellPolygons[i],color=-cellTensions[i],colormap=:plasma,colorrange=Levlims, strokecolor=(:black,1.0),strokewidth=1)
end
#Label(fig[2,1,Bottom()],"λ_"*string(n)*" = "*@sprintf("%.5E", evals[n]),fontsize = 32)

#hidedecorations!(ax22)
#hidespines!(ax22)

colsize!(fig.layout,1,Aspect(1,1.0))


Colorbar(fig[1,2],limits=colorrange=Aevlims,colormap=:plasma,flipaxis=true)
Colorbar(fig[2,2],limits=colorrange=Levlims,colormap=:plasma,flipaxis=true)


Label(fig[1,1,Bottom()],string(L"Pressure"),fontsize = 32, rotation=0)
Label(fig[2,1,Bottom()],string(L"Tension"),fontsize = 32, rotation=0)
#Label( fig[0,:],"Γ = "*string(params.γ)*", L₀ = "*string(params.L₀)*", δL = "*string(params.δL),fontsize = 32, color = (:black, 1))
resize_to_layout!(fig)
save(datadir(f,"eigenmodes","Pressures_Tensions.png"),fig)

display(fig)

In [ ]:
Peff=getPeff(params, matrices)
cellQ, cellJ=makeCellQandJ(params, matrices)
cellShearStress=getShearStress(params, matrices, cellJ)

In [ ]:

cellPolygons = makeCellPolygons(R,params,matrices)
@unpack cellAreas, cellPerimeters, cellPressures, cellTensions = matrices
Plims=(minimum(cellPressures[1:nCells]), maximum(cellPressures[1:nCells]))
Tlims=(minimum(-cellTensions[1:nCells]), maximum(-cellTensions[1:nCells]))
Pefflims=(-maximum(abs.(Peff)), maximum(abs.(Peff)))
ShStlims=(minimum(cellShearStress[1:nCells]), maximum(cellShearStress[1:nCells]))
set_theme!(figure_padding=1, backgroundcolor=(:white,1.0), font="Helvetica", fontsize=19)
fig = Figure(resolution=(1500,500))

a11=Axis(fig[1,1],aspect=DataAspect())
a21=Axis(fig[2,1],aspect=DataAspect())
a13=Axis(fig[1,3],aspect=DataAspect())
a23=Axis(fig[2,3],aspect=DataAspect())
hidedecorations!(a11)
hidespines!(a11)
hidedecorations!(a21)
hidespines!(a21)
hidedecorations!(a13)
hidespines!(a13)
hidedecorations!(a23)
hidespines!(a23)
for i=1:nCells
    poly!(a11,cellPolygons[i],color=cellPressures[i],colormap=cgrad(:Blues_9, rev=true),colorrange=Plims, strokecolor=(:black,1.0),strokewidth=1)
    poly!(a21,cellPolygons[i],color=-cellTensions[i],colormap=:plasma,colorrange=Tlims, strokecolor=(:black,1.0),strokewidth=1)
    poly!(a13,cellPolygons[i],color=Peff[i],colormap=:bwr,colorrange=Pefflims, strokecolor=(:black,1.0),strokewidth=1)
    poly!(a23,cellPolygons[i],color=cellShearStress[i],colormap=:plasma,colorrange=ShStlims, strokecolor=(:black,1.0),strokewidth=1)
end
#Label(fig[2,1,Bottom()],"λ_"*string(n)*" = "*@sprintf("%.5E", evals[n]),fontsize = 32)

#hidedecorations!(ax22)
#hidespines!(ax22)

colsize!(fig.layout,1,Aspect(1,1.0))

colsize!(fig.layout,3,Aspect(1,1.0))


Colorbar(fig[1,2],limits=colorrange=Plims,colormap=cgrad(:Blues_9, rev=true),flipaxis=true)
Colorbar(fig[2,2],limits=colorrange=Tlims,colormap=:plasma,flipaxis=true)
Colorbar(fig[1,4],limits=colorrange=Pefflims,colormap=:bwr,flipaxis=true)
Colorbar(fig[2,4],limits=colorrange=ShStlims,colormap=:plasma,flipaxis=true)

Label(fig[1,1,Bottom()],"Pressure",fontsize = 32, rotation=0)
Label(fig[2,1,Bottom()],"Tension",fontsize = 32, rotation=0)

Label(fig[1,3,Bottom()],"Effective Pressure",fontsize = 32, rotation=0)
Label(fig[2,3,Bottom()],"Shear Stress",fontsize = 32, rotation=0)
Label( fig[0,:],"Γ = "*string(params.γ)*", L₀ = "*string(params.L₀),fontsize = 32, color = (:black, 1))
resize_to_layout!(fig)
save(datadir(f,"eigenmodes","Stress.png"),fig)

display(fig)

In [ ]:
cellShapeTensors=makeShapeTensors(R,params, matrices)
circularity=getCircularity(params, cellShapeTensors)
shapeParameter=cellPerimeters./sqrt.(cellAreas)


In [ ]:
@unpack cellAreas, cellPerimeters, cellPressures, cellTensions = matrices
Aevlims=(minimum(shapeParameter[1:nCells]), maximum(shapeParameter[1:nCells]))
Levlims=(minimum(circularity[1:nCells]), maximum(circularity[1:nCells]))
set_theme!(figure_padding=1, backgroundcolor=(:white,1.0), font="Helvetica", fontsize=19)
fig = Figure(resolution=(1500,500))

a1=Axis(fig[1,1],aspect=DataAspect())
a2=Axis(fig[2,1],aspect=DataAspect())
hidedecorations!(a1)
hidespines!(a1)
hidedecorations!(a2)
hidespines!(a2)
for i=1:nCells
    poly!(a1,cellPolygons[i],color=circularity[i],colormap=:viridis,colorrange=Levlims, strokecolor=(:black,1.0),strokewidth=1)
    poly!(a2,cellPolygons[i],color=shapeParameter[i],colormap=:cividis,colorrange=Aevlims, strokecolor=(:black,1.0),strokewidth=1)
end
#Label(fig[2,1,Bottom()],"λ_"*string(n)*" = "*@sprintf("%.5E", evals[n]),fontsize = 32)

#hidedecorations!(ax22)
#hidespines!(ax22)

colsize!(fig.layout,1,Aspect(1,1.0))


Colorbar(fig[1,2],limits=colorrange=Levlims,colormap=:viridis,flipaxis=true)
Colorbar(fig[2,2],limits=colorrange=Aevlims,colormap=:cividis,flipaxis=true)


Label(fig[1,1,Bottom()],"Circularity",fontsize = 26, rotation=0)
Label(fig[2,1,Bottom()],"Shape parameter",fontsize = 26, rotation=0)
#Label( fig[0,:],"Γ = "*string(params.γ)*", L₀ = "*string(params.L₀)*", δL = "*string(params.δL),fontsize = 32, color = (:black, 1))
resize_to_layout!(fig)
save(datadir(f,"eigenmodes","circularity_shape.png"),fig)

display(fig)

In [ ]:
modeSign=zeros(2*nCells)
mode_mag_plus=zeros(2*nCells)
mode_mag_minus=zeros(2*nCells)
mode_magL_plus=zeros(2*nCells)
mode_magL_minus=zeros(2*nCells)
mode_magA_plus=zeros(2*nCells)
mode_magA_minus=zeros(2*nCells)
for n=1:2*nCells
    a=LcEevecs[1:nCells,n].*LcEevecs[nCells+1:end,n]
    bA=LcEevecs[1:nCells,n]
    bL=LcEevecs[nCells+1:end,n]
    modeSign[n]=length(a[a.>0])
    mode_mag_plus[n]=sum(a[a.>0])
    mode_mag_minus[n]=sum(a[a.<0])
    mode_magL_plus[n]=sum(bL[a.>0])
    mode_magL_minus[n]=sum(bL[a.<0])
    mode_magA_plus[n]=sum(bA[a.>0])
    mode_magA_minus[n]=sum(bA[a.<0])
end
n=LinRange(1, 2*nCells, 2*nCells)
fig = Figure()
ax=Axis(fig[1, 1], xlabel="n", ylabel="Count", title="Number of cells with the same sign A and L modes")

#scatter!(ax,n, mode_magL_plus, color=:red, label="Σ L modes, same sign", markersize=7)
scatter!(ax,n, modeSign, color=:black, label="Σ L modes, opposite", markersize=7)
# scatter!(ax,n, mode_magA_plus.+ mode_magL_plus, color=:red, label="Σ same sign", markersize=7)
# scatter!(ax,n, mode_magA_minus.+ mode_magL_minus, color=:blue, label="Σ opposite sign", markersize=7)

#vlines!(ax,2*nVerts-(2*nCells) +0.5, color=:red)
vlines!(ax,nCells+0.5, color=:red)
#fig[1, 2] = Legend(fig, ax, framevisible = false)
save(datadir(f,"eigenmodes","mode_sign.png"),fig)
fig

In [ ]:
modeSign_interior=zeros(2*nCells)
modeSign_boundary=zeros(2*nCells)
mode_mag_plus_B=zeros(2*nCells)
mode_mag_minus_B=zeros(2*nCells)
mode_mag_plus_I=zeros(2*nCells)
mode_mag_minus_I=zeros(2*nCells)
mode_magLB_plus=zeros(2*nCells)
mode_magLB_minus=zeros(2*nCells)
mode_magAB_plus=zeros(2*nCells)
mode_magAB_minus=zeros(2*nCells)
mode_magLI_plus=zeros(2*nCells)
mode_magLI_minus=zeros(2*nCells)
mode_magAI_plus=zeros(2*nCells)
mode_magAI_minus=zeros(2*nCells)
for n=1:2*nCells
    aB=LcEevecs[boundaryCells,n].*LcEevecs[boundaryCells.+100,n]
    bBA=LcEevecs[boundaryCells,n]
    bBL=LcEevecs[boundaryCells.+100,n]
    aI=LcEevecs[interiorCells,n].*LcEevecs[interiorCells.+100,n]
    bIA=LcEevecs[interiorCells,n]
    bIL=LcEevecs[interiorCells.+100,n]
    modeSign_boundary[n]=length(aB[aB.>0])
    modeSign_interior[n]=length(aI[aI.>0])
    mode_mag_plus_B[n]=sum(abs.(aB[aB.>0]))
    mode_mag_minus_B[n]=sum(abs.(aB[aB.<0]))
    mode_mag_plus_I[n]=sum(abs.(aI[aI.>0]))
    mode_mag_minus_I[n]=sum(abs.(aI[aI.<0]))
    mode_magLB_plus[n]=sum(abs.(bBL[aB.>0]))
    mode_magLB_minus[n]=sum(abs.(bBL[aB.<0]))
    mode_magAB_plus[n]=sum(abs.(bBA[aB.>0]))
    mode_magAB_minus[n]=sum(abs.(bBA[aB.<0]))
    mode_magLI_plus[n]=sum(abs.(bIL[aI.>0]))
    mode_magLI_minus[n]=sum(abs.(bIL[aI.<0]))
    mode_magAI_plus[n]=sum(abs.(bIA[aI.>0]))
    mode_magAI_minus[n]=sum(abs.(bIA[aI.<0]))
end
n=LinRange(1, 2*nCells, 2*nCells)
# fig = Figure()
# ax=Axis(fig[1, 1], xlabel="n", ylabel="count", title="sign of modes")

# # scatter!(ax,n, mode_mag_plus_I, color=:darkred, label="Interior plus", markersize=7)
# # scatter!(ax,n, mode_mag_minus_I, color=:darkblue, label="Interior minus", markersize=7)
# # scatter!(ax,n, mode_mag_plus_B, color=:red, label="Boundary plus", markersize=7)
# # scatter!(ax,n, mode_mag_minus_B, color=:blue, label="Boundary minus", markersize=7)
# scatter!(ax,n, mode_magAI_plus, color=:darkred, label="Σ A interior, same sign", markersize=7)
# scatter!(ax,n, mode_magAI_minus, color=:darkblue, label="Σ A interior, opposite sign", markersize=7)
# scatter!(ax,n, mode_magLI_plus, color=:red, label="Σ L interior, same sign", markersize=7)
# scatter!(ax,n, mode_magLI_minus, color=:blue, label="Σ L interior, opposite sign", markersize=7)

# #vlines!(ax,2*nVerts-(2*nCells) +0.5, color=:red)
# #vlines!(ax,nCells+0.5, color=:red)
# fig[1, 2] = Legend(fig, ax, framevisible = false)
# #save(datadir(f,"eigenmodes","sum_modes_sign.png"),fig)
# fig

fig = Figure()
ax=Axis(fig[1, 1], xlabel="n", ylabel="sum", title="Sum of cell mode magnitudes by sign in A and L modes")

# scatter!(ax,n, mode_mag_plus_I, color=:darkred, label="Interior same", markersize=7)
# scatter!(ax,n, mode_mag_minus_I, color=:darkblue, label="Interior opposite", markersize=7)
# scatter!(ax,n, mode_mag_plus_B, color=:red, label="Boundary same", markersize=7)
# scatter!(ax,n, mode_mag_minus_B, color=:blue, label="Boundary opposite", markersize=7)
#scatter!(ax,n, mode_magAB_plus, color=:red, label="Σ A boundary, same sign", markersize=7)
#scatter!(ax,n, mode_magAB_minus, color=:blue, label="Σ A boundary, opposite sign", markersize=7)
scatter!(ax,n, mode_magLB_plus.+mode_magAB_plus, color=:red, label="Σ  boundary, same sign", markersize=7)
scatter!(ax,n, mode_magLB_minus.+mode_magAB_minus, color=:blue, label="Σ  boundary, opposite sign", markersize=7)
#scatter!(ax,n, mode_magAI_plus, color=:darkred, label="Σ A interior, same sign", markersize=7)
#scatter!(ax,n, mode_magAI_minus, color=:darkblue, label="Σ A interior, opposite sign", markersize=7)
scatter!(ax,n, mode_magLI_plus.+mode_magAI_plus, color=:darkred, label="Σ interior, same sign", markersize=7)
scatter!(ax,n, mode_magLI_minus.+mode_magAI_minus, color=:darkblue, label="Σ interior, opposite sign", markersize=7)

#vlines!(ax,2*nVerts-(2*nCells) +0.5, color=:red)
#vlines!(ax,nCells+0.5, color=:red)
fig[1, 2] = Legend(fig, ax, framevisible = false)
#save(datadir(f,"eigenmodes","sum_modes_by_sign_interior_boundary.png"),fig)
fig

In [ ]:
n=LinRange(76, 125, 50)

In [ ]:
cells=[x for x in 1:nCells]
boundaryEdgeIndex=findall(x->x!=0,boundaryEdges)

boundaryCells=getindex.(findall(x->x!=0,Matrix(B̄)[:,boundaryEdgeIndex]), 1)
interiorCells=setdiff(cells, boundaryCells)

In [ ]:
for n=1:2*nCells
    Aintlims=(-maximum(abs.(LcEevecs[interiorCells,n])), maximum(abs.(LcEevecs[interiorCells,n])))
    Lintlims=(-maximum(abs.(LcEevecs[interiorCells.+100,n])), maximum(abs.(LcEevecs[interiorCells.+100,n])))
    Aboundlims=(-maximum(abs.(LcEevecs[boundaryCells,n])), maximum(abs.(LcEevecs[boundaryCells,n])))
    Lboundlims=(-maximum(abs.(LcEevecs[boundaryCells.+100,n])), maximum(abs.(LcEevecs[boundaryCells.+100,n])))
    set_theme!(figure_padding=1, backgroundcolor=(:white,1.0), font="Helvetica", fontsize=19)
    fig = Figure(resolution=(1500,500))

    a11=Axis(fig[1,1],aspect=DataAspect())
    a21=Axis(fig[2,1],aspect=DataAspect()) 
    a12=Axis(fig[1,3],aspect=DataAspect())
    a22=Axis(fig[2,3],aspect=DataAspect())
    hidedecorations!(a11)
    hidespines!(a11)
    hidedecorations!(a21)
    hidespines!(a21)
    hidedecorations!(a12)
    hidespines!(a12)
    hidedecorations!(a22)
    hidespines!(a22)
    for i in interiorCells
        poly!(a11,cellPolygons[i],color=[LcEevecs[1:nCells,n][i]],colormap=:bwr,colorrange=Aintlims, strokecolor=(:black,1.0),strokewidth=1)
        poly!(a21,cellPolygons[i],color=[LcEevecs[nCells+1:2*nCells,n][i]],colormap=:bwr,colorrange=Lintlims, strokecolor=(:black,1.0),strokewidth=1)
    end
    for i in boundaryCells
        poly!(a12,cellPolygons[i],color=[LcEevecs[1:nCells,n][i]],colormap=:bwr,colorrange=Aboundlims, strokecolor=(:black,1.0),strokewidth=1)
        poly!(a22,cellPolygons[i],color=[LcEevecs[nCells+1:2*nCells,n][i]],colormap=:bwr,colorrange=Lboundlims, strokecolor=(:black,1.0),strokewidth=1)
    end
    Label(fig[2,1,Bottom()],"λ_"*string(n)*" = "*@sprintf("%.5E", LcEevals[n]),fontsize = 32)

    #hidedecorations!(ax22)
    #hidespines!(ax22)

    colsize!(fig.layout,1,Aspect(1,1.0))
    colsize!(fig.layout,3,Aspect(1,1.0))


    Colorbar(fig[1,2],limits=colorrange=Aintlims,colormap=:bwr,flipaxis=true)
    Colorbar(fig[2,2],limits=colorrange=Lintlims,colormap=:bwr,flipaxis=true)
    
    Colorbar(fig[1,4],limits=colorrange=Aboundlims,colormap=:bwr,flipaxis=true)
    Colorbar(fig[2,4],limits=colorrange=Lboundlims,colormap=:bwr,flipaxis=true)


    Label(fig[1,1,Left()],string(L"Area"),fontsize = 32, rotation=π/2)
    Label(fig[2,1,Left()],string(L"Perimeter"),fontsize = 32, rotation=π/2)
    #Label( fig[0,:],"Γ = "*string(params.γ)*", L₀ = "*string(params.L₀)*", δL = "*string(params.δL),fontsize = 32, color = (:black, 1))
    resize_to_layout!(fig)
    save(datadir(f,"eigenmodes","eigenmodes_split_edge_LcE$(@sprintf("%03d", n)).png"),fig)

    #display(fig)
end

In [ ]:
fev=Glob.glob("relax_disordered_T1/*3.71*/eigenmodes/H_eigenvalues.csv","C:\\Users\\v35431nc\\Documents\\VM_code\\VertexModel\\data\\sims" )[1]

In [ ]:
H_ev_Γ_00289=readdlm(fev[1],Float64)